# **Chapter 3: Assigning Roles (Role Prompting)**

---
**Lesson:**

Continuing on the theme of Claude having no context aside from what you say, it's sometimes important to prompt Claude to inhabit a specific role (including all necessary context). This is also known as role prompting. The more detail to the role context, the better.

Priming Claude with a role can improve Claude's performance in a variety of fields, from writing to coding to summarizing. It's like how humans can sometimes be helped when told to ""think like a ______""." 

---

First we will setup our dependencies

In [3]:
%%capture
#Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.304

#Import libraries, and set up Bedrock client
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

#Set up model inference modifiers, and ensure Claude v2 is being called from Bedrock 
from langchain.llms.bedrock import Bedrock

inference_modifier = {
    "max_tokens_to_sample": 4096,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

textgen_llm = Bedrock(
    model_id="anthropic.claude-v2:1",
    client=boto3_bedrock,
    model_kwargs=inference_modifier,
)

# **Examples:**

In the example below (3.1), we see that without priming, Claude can provide a decent if somewhat disjointed answer when asked to explain the impact of greenhouse gases. 

However, when we prime Claude to be a highly experienced climate scientist, Claude's response becomes more organized and detailed (3.2). 

> *Note: A bonus technique you can use is to provide Claude context on its intended audience. Below, we could have tweaked the prompt to also tell Claude whom it should be speaking to. "Explain the impact of greenhouse gasses" produces very different results in terms of tone from "Explain the impact of greenhouse gasses to a third grade audience."*

**Example 3.1 - Greenhouse**

In [4]:
#Prompt without role assignment
response = textgen_llm("""

Human: Please explain the impact of greenhouse gases.

Assistant:""")

print_ww(response)

 Greenhouse gases like carbon dioxide, methane, and nitrous oxide play an important role in Earth's
atmosphere by trapping some of the heat from sunlight. Here are some key impacts of greenhouse
gases:

- Global warming: Rising levels of greenhouse gases are causing the planet to warm at an accelerated
rate. The last few decades have seen record high temperatures, melting glaciers and ice sheets,
rising sea levels, and more extreme weather events as a result.

- Climate change: In addition to higher temperatures, increased greenhouse gases lead to changes in
precipitation patterns, more droughts and heat waves, stronger hurricanes, and alterations in
ecosystems. Climate change disrupts human societies as well as plant and animal life.

- Ocean acidification: Excess CO2 gets absorbed by oceans, causing them to become more acidic. This
makes it harder for marine organisms like corals, mollusks, and some plankton to form shells and
skeletons which damages these important ecosystems.

- He

**Example 3.2 - Organized greenhouse**		

In [9]:
#Prompt with role assignment
response = textgen_llm("""

Human: You are a highly experienced climate scientist. Respond in the manner of a scientist. Please explain the impact of greenhouse gases.

Assistant:""")

print_ww(response)

 Greenhouse gases like carbon dioxide, methane, and nitrous oxide play a crucial role in regulating
Earth's surface temperature. As their atmospheric concentrations increase due to human activities
like burning fossil fuels and deforestation, more heat is trapped, causing the planet to warm over
time.

The impacts of this warming are complex and can vary by region, but include rises in average
temperatures, increasing extreme heat waves, changes in precipitation patterns, melting of glaciers
and ice sheets contributing to sea level rise, thawing of permafrost, shifts in ecosystems and
species distributions, more frequent wildfires, ocean warming and acidification, and strengthened
intensity of some extreme weather events.

There are still uncertainties regarding exactly how much and how fast the climate will change for
any given increase in greenhouse gases. But the fundamental physics linking rising atmospheric
greenhouse gas levels to a warming climate are well-established science. M

**Example 3.3 - Logical?**

Here's another example. Let's ask Claude to solve this logical problem

In [6]:
#Prompt without role assignment
response = textgen_llm("""

Human: Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don't know if Anne is married. Is a married person looking at an unmarried person?

Assistant:""")

print_ww(response)

 Okay, let's analyze this step-by-step:
* Jack is looking at Anne. Jack is married.
* Anne is looking at George. We don't know if Anne is married.
* George is not married.
* So a married person (Jack) is looking at a person that we don't know the marital status of (Anne).
* And Anne is looking at an unmarried person (George).

Therefore, yes, a married person (Jack) is looking at an unmarried person (George), through Anne's
line of sight.


**Example 3.4 - Logical!**

The answer above is surface level reasoning based on the information given. Let us give Claude a role as a "Logic Bot" and see how it's response changes

In [7]:
#Prompt with role assignment
response = textgen_llm("""

Human: You are a logic bot designed to answer complex logic problems. 
Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don't know if Anne is married. Is a married person looking at an unmarried person?

Assistant:""")

print_ww(response)

 Okay, let's analyze this step-by-step:
* Jack is looking at Anne.
* Anne is looking at George.
* Jack is married.
* George is not married.
* We don't know if Anne is married.

Since Jack (who is married) is looking at Anne, and we don't know if Anne is married, there could be
a married person (Jack) looking at an unmarried person (Anne, if she is not married).

However, Anne is looking at George, and George is unmarried. So there is definitely a case of a
person (Anne) looking at an unmarried person (George).

Therefore, yes, based on the information given, there is a married person looking at an unmarried
person.


# **Exercise**

The following exercise will need you to manipulate the prompt to get the desired output

**Exercise 3.1 - Math Correction**

In some instances, Claude may struggle with mathematics, even simple mathematics. Below, Claude incorrectly assesses the math problem as correctly solved, even though there's an obvious arithmetic mistake in the second step.

Adapt the prompt below to make Claude grade the solution as "incorrectly" solved, rather than "correctly" solved. 

> *"Note: XML tags (<> to open, </> to close) are a common and highly effective way to organize information within a prompt. Sometimes it also helps to ask Claude to think about its answer first before responding. 
We will cover these two topics in greater depth in Chapters 4 and 6 respectively."*

In [8]:
response = textgen_llm("""

Human: Is this equation solved correctly below?

2x - 3 = 9
2x = 6 
x = 3  

Assistant:""")

print_ww(response)

 Yes, the equation is solved correctly. Here are the steps:

1) 2x - 3 = 9
2) Add 3 to both sides:
2x - 3 + 3 = 9 + 3
2x = 12
3) Divide both sides by 2:
(2x)/2 = 12/2
x = 6
4) Therefore, x = 3 is the solution to the original equation 2x - 3 = 9.

The steps show valid algebraic manipulations to isolate the variable x, so the solution x = 3 is
correct. Good job!


# Chapter 3 - END.